In [1]:
import pandas as pd
import numpy as np

### 데이터 로드

In [16]:
df = pd.read_csv('..\data\Preprocessed_Bank_Customer_Churn_Prediction.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Playdata\AppData\Local\Temp\ipykernel_9088\1384499252.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('..\data\Preprocessed_Bank_Customer_Churn_Prediction.csv')
C:\Users\Playdata\AppData\Local\Temp\ipykernel_9088\1384499252.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('..\data\Preprocessed_Bank_Customer_Churn_Prediction.csv')


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\Preprocessed_Bank_Customer_Churn_Prediction.csv'

In [15]:
df 

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,2,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,2,1,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64,0
9996,516,0,0,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,1,36,7,0.00,1,0,1,42085.58,1
9998,772,1,0,42,3,75075.31,2,1,0,92888.52,1


### 머신 러닝()